In [17]:
import json

# key = input("이름을 입력하세요: ")
key = "정윤"

filename = "message_1.json"
filename = "message_111.json"
filename = "message_123.json"

# 원본 문자열
raw_chat = ""
with open(filename, 'r') as f:
    raw_chat = f.read()
    raw_chat = json.loads(raw_chat)
# print(raw_chat)

def decode_utf8_escaped(s):
    return s.encode('latin1').decode('utf-8')

print(raw_chat)
messages = raw_chat["messages"]
for i in range(len(messages)):
    messages[i]['sender_name'] = decode_utf8_escaped(messages[i]['sender_name'])
    if 'content' in messages[i]:
        messages[i]['content'] = decode_utf8_escaped(messages[i]['content'])

{'participants': [{'name': 'á\x84\x92á\x85®á\x86«á\x84\x91á\x85\xad'}, {'name': 'í\x97\x88ì\xa0\x95ì\x9c¤'}], 'messages': [{'sender_name': 'í\x97\x88ì\xa0\x95ì\x9c¤', 'timestamp_ms': 1740395111110, 'content': 'ì\x95\x8cê²\x8cë\x94°', 'is_geoblocked_for_viewer': False, 'is_unsent_image_by_messenger_kid_parent': False}, {'sender_name': 'í\x97\x88ì\xa0\x95ì\x9c¤', 'timestamp_ms': 1740395103858, 'content': 'ã\x85\x8bã\x85\x8bã\x85\x8bã\x85\x8bã\x85\x8bã\x85\x8b', 'is_geoblocked_for_viewer': False, 'is_unsent_image_by_messenger_kid_parent': False}, {'sender_name': 'á\x84\x92á\x85®á\x86«á\x84\x91á\x85\xad', 'timestamp_ms': 1740395047739, 'content': 'ì\x95\x8cë\xa0¤ì¤\x98', 'is_geoblocked_for_viewer': False, 'is_unsent_image_by_messenger_kid_parent': False}, {'sender_name': 'á\x84\x92á\x85®á\x86«á\x84\x91á\x85\xad', 'timestamp_ms': 1740395044131, 'content': 'ë\x82\xa0ì§\x9cë\x82\x98ì\x98¤ë©´', 'is_geoblocked_for_viewer': False, 'is_unsent_image_by_messenger_kid_parent': False}, {'sender_name'

In [ ]:
from collections import defaultdict
import re
from datetime import datetime
import os

def normalize_message(content):
    content = content.strip()
    if '이모티콘' in content:
        return '(이모티콘)'
    if '사진' in content:
        return '(사진)'
    # 감탄사, 말줄임표 치환
    content = re.sub(r'(!|ㄷ|ㅎ|ㅋ|ㅠ|ㅜ|\?|헐|하|헤|호|흐|허|와우|오오|진짜){2,}', r'\1\1', content)
    content = re.sub(r'\.{3,}', '...', content)
    # 링크 치환
    content = re.sub(r'http[s]?://\S+', '(링크)', content)
    # 계좌번호 (10~14자리)
    content = re.sub(r'\b\d{10,14}\b', '(계좌번호)', content)
    # 휴대폰/집전화번호
    content = re.sub(r'\b01[016789]-?\d{3,4}-?\d{4}\b', '(전화번호)', content)
    content = re.sub(r'\b(0\d{1,2}-?\d{3,4}-?\d{4})\b', '(전화번호)', content)
    # 카드번호
    content = re.sub(r'\b\d{4}-\d{4}-\d{4}-\d{4}|\d{16}\b', '(카드번호)', content)
    # 이메일 주소 치환
    content = re.sub(r'\b[\w\.-]+@[\w\.-]+\.\w+\b', '(이메일)', content)
    return content

# ---- 메인 로직 ----
sorted_data = sorted(messages, key=lambda x: x['timestamp_ms'])
grouped_by_date = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

for item in sorted_data:
    if 'content' in item and "sent an attachment" not in item['content']:
        timestamp = datetime.fromtimestamp(item['timestamp_ms'] / 1000.0)
        date_str = timestamp.strftime("%Y-%m-%d")
        time_str = timestamp.strftime("%H:%M")
        sender = "나" if key in item['sender_name'] else "상대방"
        content = normalize_message(item['content'])
        grouped_by_date[date_str][time_str][sender].append(content)

formatted_lines = []
for date in sorted(grouped_by_date):
    formatted_lines.append(f"[{date}]")
    for time in sorted(grouped_by_date[date]):
        for sender in grouped_by_date[date][time]:
            merged_msg = " | ".join(grouped_by_date[date][time][sender])
            formatted_lines.append(f"{time} - {sender}: {merged_msg}")
    formatted_lines.append("")

formatted_chat = "\n".join(formatted_lines).strip()

os.makedirs('processed', exist_ok=True)
with open(f'processed/{filename}.txt', 'w', encoding='utf-8') as f:
    f.write(formatted_chat)
print(formatted_chat)


[2024-09-12]
18:10 - 상대방: 으 사랑꾼
18:11 - 상대방: 부럽다!
19:14 - 나: ㅋㅋ오글겅려?
19:52 - 상대방: 노 | 보기좋아 ㅎㅎ
21:57 - 나: ㅋㅋ | 고맙다

[2025-02-24]
18:03 - 상대방: 정윤 왜케 달달해 결혼 언제
18:21 - 나: ㅋㅋ | 아직일년도 더남았다
20:04 - 상대방: 날짜나오면 | 알려줘
20:05 - 나: ㅋㅋ | 알게따
